# 00000: Model validation and optimization
Authors: Tobias G. Mueller, Mark A. Buckner
Last modified: 4 Dec 2024
Contact: __________

**Summary**: Here, we use the tiled model predictions as well as validate it against a labeled ground truth.
We then calculate the confidence threshold that optimizes f1 score and filter out main model predictions are that level.


This script outputs 
- an optimized model predicting on the whole orthomosaic
- model performance metrics

The data used in this script was generated in:
    `AIggregation/notebooks/02_predict_and_tile.ipynb`
    - labelstudio, labeling ground truth of the output from above script

Before running this script you must have ground truthed the tiled testset output from `02_predict_and_tile.ipynb` using label studio 

In [1]:
#imports 
import os
import fiftyone as fo
from fiftyone import ViewField as F
import scipy
import fiftyone as fo
import yaml

# first check the wd is not notebooks but the main folder
print("cwd is", os.getcwd())

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")
    print("cwd changed to", os.getcwd())


cwd is /home/tmueller/github/AIggregation/notebooks
cwd changed to /home/tmueller/github/AIggregation


First we import the tiled predictions and corresponding ground truths that were annotated and remove any split boxes that were cropped during the tiling.

In [2]:

# set paths
# -------------------------------------------------------------------------------------------------------------------- #
gt_directory = "datasets/testset/groundtruth_testset"       # Directory with labelstudio groundtruth export extracted. Should contain "images" and "labels" folders
tiled_test_directory = "datasets/testset/tiled_testset"     # target folder for tile script ("target_path" in previous notebook)
# -------------------------------------------------------------------------------------------------------------------- #



# rename label studio outputs to have same name as fiftyone outputs (label studio adds long numeric strings upon export)
for root, dirs, files in os.walk(gt_directory):
    for file in files:
        oldname = os.path.join(root,file)
        if "-" in file:
            newname = file.split("-")[1]
            os.rename(oldname, os.path.join(root,newname))


# create a YAML file in the ground_truth folder 
folder = gt_directory.split('/')[-1] # get end of string (folder name)

data = {
    'names':{
    0: "nest"},
    'path': os.path.join("..", folder),
    'val': "./images/"
        }
with open((os.path.join(gt_directory,"data.yaml")), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False, sort_keys=False)




# import test tiles into fiftyone
datasettest = fo.Dataset.from_dir(
    dataset_type=fo.types.YOLOv5Dataset,
    yaml_path = os.path.join(tiled_test_directory, "data.yaml"),
    label_field= "prediction"
)

# import groundtruthed tiles into fiftyone
dataset_ground = fo.Dataset.from_dir(
    dataset_type=fo.types.YOLOv5Dataset,
    yaml_path = os.path.join(gt_directory,"data.yaml"),
    label_field= "ground_truth"
)

# merge the datasets, merging images/labels with the same file name
key_fcn = lambda sample: os.path.basename(sample.filepath)

datasettest.merge_samples(dataset_ground, key_fcn=key_fcn)



# delete any oblong box predictions
# these are created by splitting tiles through the middle of an existing bounding box. Nest predictions should not be oblong

# Compute the dimensions of each bounding box in pixels
box_width, box_height = F("bounding_box")[2], F("bounding_box")[3]

# remove detections where one side of the box is greater than 2.5 time the other
def get_label_fields(sample_collection):
    """Get the (detection) label fields of a Dataset or DatasetView."""
    label_fields = list(
        sample_collection.get_field_schema(embedded_doc_type=fo.Detections).keys()
    )
    return label_fields

for lf in get_label_fields(datasettest):
    datasettest_keep = datasettest.filter_labels(
    "prediction", (box_height < (box_width*2)) & (box_width < (box_height*2)), only_matches=False)


# create a subset view of just boxes that were removed for visualization 
removed_boxes=datasettest.select_fields("prediction").filter_labels(
    "prediction", (box_height > (box_width*2)) | (box_width > (box_height*2)) 
)
#session = fo.launch_app(removed_boxes)



# view that this worked. Ideally each nest should have a "predict" label and a "ground_truth" label
session = fo.launch_app(datasettest_keep)



 100% |█████████████████| 116/116 [188.1ms elapsed, 0s remaining, 616.6 samples/s]      
 100% |█████████████████| 116/116 [132.8ms elapsed, 0s remaining, 884.7 samples/s] 
Indexing dataset...
 100% |█████████████████| 116/116 [59.9ms elapsed, 0s remaining, 1.9K samples/s] 
Merging samples...
 116 [324.2ms elapsed, ? remaining, 359.7 samples/s] 


We can then use the ground truths to validate our model predictions. 

Each detection made by the model has a confidence associated with it. We will filter at the confidence threshold that leads to the highest F1 score

In [23]:
# set parameters
# -------------------------------------------------------------------------------------------------------------------- #
dataset = datasettest_keep      # filtered dataset 
prediction = "prediction"       # prediction name
gt = "ground_truth"             # ground truth name
lb = .1                         # lower confidence to check between
ub = .99                        # upper confidence to check between
iou = 0.4                       # what IOU to use for FN and FP detections
# -------------------------------------------------------------------------------------------------------------------- #



# define function to calculate the f1 score of a model
def calculate_f1(conf, dataset, prediction, gt):
    conf_view = dataset.filter_labels(prediction, F("confidence") >= conf)
    results = conf_view.evaluate_detections(prediction,
        gt_field= gt,
        eval_key="eval",
        missing="fn",
        iou = iou)

    fp = sum(conf_view.values("eval_fp"))
    tp = sum(conf_view.values("eval_tp"))
    fn = sum(conf_view.values("eval_fn"))

    f1 = tp/(tp+0.5*(fp+fn))

    return -1.0*f1  #make output negative to use fminbound in following function

calculate_f1(dataset=dataset, conf=.5, prediction=prediction, gt=gt)


# define function find the optimal output of the above function
def optimize_conf(lb, ub, dataset, gt, prediction):

    res = scipy.optimize.fminbound(
                    func=calculate_f1,
                    x1=lb,
                    x2=ub,
                    args=(dataset, prediction, gt),
                    xtol=0.01,
                    full_output=True
    )

    best_conf, f1val, ierr, numfunc = res
    maxf1 = -1.0*f1val
    print("\n \n     best f1          at confidence")
    print(maxf1, best_conf)
    return maxf1, best_conf


# save second output, best_conf, to a variable for use in filtering predictions
bestconf = optimize_conf(lb=lb, ub=ub, dataset=dataset, gt=gt, prediction=prediction)[1]


# filter and evaluate model performance at best conf threshold
high_f1_view = datasettest_keep.filter_labels("prediction", F("confidence") > bestconf, only_matches=False)



Evaluating detections...
 100% |█████████████████| 101/101 [514.3ms elapsed, 0s remaining, 196.4 samples/s]      
Evaluating detections...
 100% |█████████████████| 101/101 [403.5ms elapsed, 0s remaining, 250.3 samples/s]      
Evaluating detections...
 100% |███████████████████| 99/99 [390.4ms elapsed, 0s remaining, 253.6 samples/s]      
Evaluating detections...
 100% |█████████████████| 101/101 [393.9ms elapsed, 0s remaining, 256.4 samples/s]      
Evaluating detections...
 100% |█████████████████| 103/103 [411.9ms elapsed, 0s remaining, 250.0 samples/s]      
Evaluating detections...
 100% |█████████████████| 101/101 [398.5ms elapsed, 0s remaining, 253.4 samples/s]      
Evaluating detections...
 100% |█████████████████| 101/101 [401.2ms elapsed, 0s remaining, 251.8 samples/s]      
Evaluating detections...
 100% |█████████████████| 101/101 [403.6ms elapsed, 0s remaining, 250.3 samples/s]      
Evaluating detections...
 100% |█████████████████| 101/101 [400.6ms elapsed, 0s remainin

Now that we have our model filtered to optimized F1, we can evaluate its performance

In [24]:
results_f1 = high_f1_view.evaluate_detections(
    prediction,
    gt_field=gt,
    eval_key="prediction",
    missing = "fn",
    iou = iou,
    compute_mAP=True
)

# print performance results
print(results_f1.mAP())
results_f1.print_report()




Evaluating detections...
 100% |█████████████████| 116/116 [408.8ms elapsed, 0s remaining, 283.8 samples/s]      
Performing IoU sweep...
 100% |█████████████████| 116/116 [453.4ms elapsed, 0s remaining, 255.8 samples/s]      
0.36663672271526565
              precision    recall  f1-score   support

        nest       0.91      0.75      0.82       309

   micro avg       0.91      0.75      0.82       309
   macro avg       0.91      0.75      0.82       309
weighted avg       0.91      0.75      0.82       309



We can then view the false negatives and false positives to get a better understanding of how the model is performaing

In [ ]:
#view false postivies

fp_view = high_f1_view.to_evaluation_patches(eval_key="prediction").match(F("type")=="fp").sort_by("prediction.detection.confidence")

session.view = fp_view.view()

In [ ]:
#view false negatives

fn_view = high_f1_view.to_evaluation_patches(eval_key="prediction").match(F("type")=="fn").sort_by("prediction.detection.confidence")

session.view = fn_view.view()

Lastly we will filter our full model detections at the optimized confidence threshold and export the detections

In [ ]:
# set paths
# -------------------------------------------------------------------------------------------------------------------- #
full_prediction_directory = "datasets/export_predictions/temp"      # directory with model detections (created in 02_predict_and_tile.ipynb)
full_export_directory = "datasets/export_predictions/final"         # directory to output optimized detections
# -------------------------------------------------------------------------------------------------------------------- #


# read in full sahi predictions
dataset_final = fo.Dataset.from_dir(
    dataset_type=fo.types.YOLOv5Dataset,
    yaml_path = os.path.join(full_prediction_directory, "data.yaml"),
    label_field= "prediction"
)

# filter at optimal confidence
dataset_full_f1 = dataset_final.filter_labels("prediction", F("confidence") > bestconf, only_matches=False)

# export as final prediction
dataset_full_f1.export(
        export_dir=full_export_directory,
        dataset_type=fo.types.YOLOv5Dataset,
        label_field="prediction",
        include_confidence=True
    )

We now have optimized nest detections across our whole orthomosaic as well as metrics for how well our model is performing. 

The model detections can now be georeferenced in `04_georeference.ipynb` 